In [94]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score

import pandas as pd

## Get data ready

In [47]:
df_train_orig = pd.read_csv('data/Train_housing.csv')

In [48]:
df_test_orig = pd.read_csv('data/Test_housing.csv')

In [9]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
# X = df_numeric[['YearMade']]
# y = df_numeric['SalePrice']

# X_train, X_test, y_train, y_test = train_test_split(X, y)

## Initiniate pipeline

In [3]:
'''
# feature = ('feature_union', feature_union)
scaler = ('scaler', StandardScaler())
model = ('lrclf', LogisticRegression())

steps = [
   # feature,
    scaler,
    model
]
pipeline = Pipeline(steps)
pipeline.fit(X, y)
'''

"\n# feature = ('feature_union', feature_union)\nscaler = ('scaler', StandardScaler())\nmodel = ('lrclf', LogisticRegression())\n\nsteps = [\n   # feature,\n    scaler,\n    model\n]\npipeline = Pipeline(steps)\npipeline.fit(X, y)\n"

In [6]:
## It takes a list of tuples as parameter
## To check the exact params keys:
# pipeline.get_params().keys()
'''
grid = {'gbr__learning_rate': [0.1, 0.05, 0.02],
        'gbr__max_depth': [2, 4, 6],
        'gbr__min_samples_leaf': [1, 2, 5, 10],
        'gbr__max_features': [1.0, 0.3, 0.1],
        'gbr__n_estimators': [100],
        'gbr__random_state': [1]}

# feature = ('feature_union', feature_union)
scaler = ('scaler', StandardScaler())
model = ('gbr', GradientBoostingRegressor())

steps = [
   # feature,
    scaler,
    model
]
pipeline = Pipeline(steps)

gridsearch = GridSearchCV(pipeline, 
                          grid,
                          n_jobs=-1,
                          scoring='mean_squared_error')
gridsearch.fit(X_train, y_train)
best_model = gridsearch.best_estimator_
cross_val_score(best_model, X_train, y_train, cv = 11, scoring = 'mean_squared_error').mean()
'''

"\ngrid = {'gbr__learning_rate': [0.1, 0.05, 0.02],\n        'gbr__max_depth': [2, 4, 6],\n        'gbr__min_samples_leaf': [1, 2, 5, 10],\n        'gbr__max_features': [1.0, 0.3, 0.1],\n        'gbr__n_estimators': [100],\n        'gbr__random_state': [1]}\n\n# feature = ('feature_union', feature_union)\nscaler = ('scaler', StandardScaler())\nmodel = ('gbr', GradientBoostingRegressor())\n\nsteps = [\n   # feature,\n    scaler,\n    model\n]\npipeline = Pipeline(steps)\n\ngridsearch = GridSearchCV(pipeline, \n                          grid,\n                          n_jobs=-1,\n                          scoring='mean_squared_error')\ngridsearch.fit(X_train, y_train)\nbest_model = gridsearch.best_estimator_\ncross_val_score(best_model, X_train, y_train, cv = 11, scoring = 'mean_squared_error').mean()\n"

In [55]:
def drop_NaN_cols(df, threshold):
    drop_cols = []
    for c in df.columns:
        if (1.*sum(df[c].isnull()))/df.shape[0] > threshold:
            drop_cols.append(c)
            df.drop(c, axis=1, inplace=True)
    return df, drop_cols

In [56]:
df_train, drop_cols = drop_NaN_cols(df_train_orig, 1./6)

In [54]:
check_nan(df_train, df_train.columns)

MasVnrType  has  8 null/NAN values
MasVnrArea  has  8 null/NAN values
BsmtQual  has  37 null/NAN values
BsmtCond  has  37 null/NAN values
BsmtExposure  has  38 null/NAN values
BsmtFinType1  has  37 null/NAN values
BsmtFinType2  has  38 null/NAN values
Electrical  has  1 null/NAN values
GarageType  has  81 null/NAN values
GarageYrBlt  has  81 null/NAN values
GarageFinish  has  81 null/NAN values
GarageQual  has  81 null/NAN values
GarageCond  has  81 null/NAN values


True

In [57]:
df_test = df_test_orig.drop(drop_cols, axis=1)

In [58]:
def brutal_fillna(df):
    for c in df.columns:
        if sum(df[c].isnull())>0:
            cond = df[c].notnull()
            if df[c].dtype=='object':
                new_val = df[cond].groupby([c])[c].count().sort_values(ascending=False)[0]
            else:
                new_val = df[cond][c].mean()
            df[c]=df[c].fillna(new_val)
    return df

In [59]:
df_train = brutal_fillna(df_train)
df_test = brutal_fillna(df_test)

In [60]:
check_nan(df_train, df_train.columns)

True

In [61]:
check_nan(df_test, df_test.columns)

True

In [101]:
pd.set_option('display.max_columns', None)

In [102]:
df_train.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [103]:
df_train['House_Age'] = df_train['YrSold'] - df_train['YearBuilt']

In [110]:
features = ['MSSubClass','LotArea','House_Age','OverallQual','OverallCond','GrLivArea']
X = df_train[features]
y = df_train['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
## To check the exact params keys:
# pipeline.get_params().keys()
'''
grid = {'gbr__learning_rate': [0.1, 0.05, 0.02],
        'gbr__max_depth': [2, 4, 6],
        'gbr__min_samples_leaf': [1, 2, 5, 10],
        'gbr__max_features': [1.0, 0.3, 0.1],
        'gbr__n_estimators': [100],
        'gbr__random_state': [1]}

feature = ('feature_union', feature_union)
scaler = ('scaler', StandardScaler())
model = ('gbr', GradientBoostingRegressor())

steps = [
   # feature,
    scaler,
    model
]
pipeline = Pipeline(steps)

gridsearch = GridSearchCV(pipeline, 
                          grid,
                          n_jobs=-1,
                          scoring='mean_squared_error')
gridsearch.fit(X_train, y_train)
best_model = gridsearch.best_estimator_
'''

"\ngrid = {'gbr__learning_rate': [0.1, 0.05, 0.02],\n        'gbr__max_depth': [2, 4, 6],\n        'gbr__min_samples_leaf': [1, 2, 5, 10],\n        'gbr__max_features': [1.0, 0.3, 0.1],\n        'gbr__n_estimators': [100],\n        'gbr__random_state': [1]}\n\nfeature = ('feature_union', feature_union)\nscaler = ('scaler', StandardScaler())\nmodel = ('gbr', GradientBoostingRegressor())\n\nsteps = [\n   # feature,\n    scaler,\n    model\n]\npipeline = Pipeline(steps)\n\ngridsearch = GridSearchCV(pipeline, \n                          grid,\n                          n_jobs=-1,\n                          scoring='mean_squared_error')\ngridsearch.fit(X_train, y_train)\nbest_model = gridsearch.best_estimator_\n"